# Transfering priority values to the putative protected area network

The original data is a shapefile with the priority of each cell and information of the country. The R script rescales the values within a country from 0 to 255. Then in arcgis pro the shapefile is converted to a raster using the rescaled value. The cell size and extent of the raster is determined by a raster that shows the global putative protected area. Finally, using a constant version of the global putative protected area the pixels that are not part of the global putative protected area are removed. 
#### Note
The R bit can probably be done in arcgis pro using SQL. 

## Processing the data by country in R


```R
library(foreign)
library(plyr)
library(dplyr)
d = read.dbf("Priority_ranks_by_country_20200723/Priority_ranks_by_country_20200723.dbf")

d_range = d %>% 
  group_by(GID_0) %>% 
  summarise(min_pr = min(GID_PR_all), max_pr = max(GID_PR_all), count = n()) %>% 
  right_join(d) %>% 
  mutate(plasma_val = ifelse(count>2,(((GID_PR_all - min_pr) / (max_pr - min_pr)) * 255), 255))   
  write.csv(d_range, "Priority_ranks_by_country_20200723_rescale.csv")
 ```

## From shapefile to raster in ArcGIS pro

In [ ]:
# this took a really long time (45 mins), probably because of not indexing the csv
#arcpy.management.JoinField("Priority_ranks_by_country_20200723","ID", 
                           "Priority_ranks_by_country_20200723_rescale.csv", "ID", 
                           "plasma_val")

In [25]:
outRaster = arcpy.conversion.PolygonToRaster("Priority_ranks_by_country_20200723", 
                                             "plasma_val_1", 
                                             r"C:\Users\Viz1\Documents\ArcGIS\Projects\PrioritisationMOL\PrioritisationMOL.gdb\Priority_ranks_by_country_20200723_rescale_PolygonToRaster", "CELL_CENTER", "NONE", 
                                             r"C:\Users\Viz1\Documents\ArcGIS\Projects\PrioritisationMOL\LP_min_HM_720_WDPAJan2020_20200212.tif")

In [26]:
pa_constant_Raster = arcpy.Raster( r"C:\Users\Viz1\Documents\ArcGIS\Projects\PrioritisationMOL\PrioritisationMOL.gdb\putative_pa_constant")

In [29]:
output_raster = outRaster * pa_constant_Raster
output_raster.save(f"C:/Users/Viz1/Documents/ArcGIS/Projects/PrioritisationMOL/PrioritisationMOL.gdb/raster_putative_rescale_countries")